#### TFIDF Recommender System
Since the results for the Bag of Words model were underwhelming, I decided to try a TFIDF approach. The reasoning for the text normalization and parameters are the same, but TFIDF allows us to give weight to words, so unique words specific to the queries are ranked higher, and words that appear in more documents are lower. For example, the query, "musical instrument store" will rank "musical" and "instrument" higher, while store will get a lower score since it appears more often in the text.

In [296]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [263]:
stop_words = set(stopwords.words("english"))

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n\\b") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 


def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers (punctuation, curly brackets etc).
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(description, stop_words, normalization):
    
    if normalization == 'lemmatize':
        # tokenize and lemmatize text
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(w) for w in word_tokenize(description)]
        
    elif normalization == 'stem':
        # tokenize and stem text
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(w) for w in word_tokenize(description)]
    
   # remove tokens length of 2 or below and make all lowercase and remove stop words
    tokens = [w.lower() for w in tokens if (w.lower() not in stop_words) and (len(w) > 2) and(w.isalpha())]
    
    return tokens    
    
def process_query(query, normalization):
    stop_words = set(stopwords.words("english"))
    
    return tokenizer(clean_text(query), stop_words, normalization)

In [264]:
# code example taken from https://towardsdatascience.com/build-a-text-recommendation-system-with-python-e8b95d9f251c
def retrieve_top_n(m, max_docs):
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score \
    mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:max_docs]  
    return best_index

In [265]:
class TfidfRecommenderSystem:
    def __init__(self, docs, alpha=1.0, beta=0.75, gamma=0.15):
        self.alpha, self.beta, self.gamma = alpha, beta, gamma
        
        # create a doc-term matrix out of our doc collection
        self.vec = TfidfVectorizer()
        doc_term_mat = self.vec.fit_transform([" ".join(docs[doc_id]) for doc_id in docs])
        result = doc_term_mat
        
        self.q_vecs = {}
        
        self.doc_vecs = result # document vectors in a matrix
        
    def retrieve_docs(self, query, max_docs=10, normalization='lemmatize'):
        query = ' '.join(process_query(query, normalization))
        
        if query not in self.q_vecs:
            q_vec = self.vec.transform([query])
            self.q_vecs[query] = q_vec
        
        ret_docs = {}
        
        mat = cosine_similarity(self.q_vecs[query], self.doc_vecs)
        best_index = retrieve_top_n(mat, max_docs=max_docs)
        
        return best_index

In [266]:
df = pd.read_pickle(r'assets/processed_df.pkl')
docs = dict(zip(df['naics'], df['lemmatized']))

In [267]:
tfidf_model = TfidfRecommenderSystem(docs)

In [268]:
best_index = tfidf_model.retrieve_docs('library', normalization='lemmatize')

In [269]:
naics_titles = pd.read_excel('assets/6-digit_2017_Codes.xlsx')
naics_titles['naics'] = naics_titles['naics'].astype(str)

In [270]:
df = df.merge(naics_titles, on='naics', how='outer')

In [271]:
df.iloc[best_index][['naics', 'title']]

,naics,title
719,519120,Libraries and Archives
721,519190,All Other Information Services
703,512199,Other Motion Picture and Video Industries
474,337127,Institutional Furniture Manufacturing
718,519110,News Syndicates
470,337121,Upholstered Household Furniture Manufacturing
472,337124,Metal Household Furniture Manufacturing
471,337122,Nonupholstered Wood Household Furniture Manufa...
473,337125,Household Furniture (except Wood and Metal) Ma...
720,519130,Internet Publishing and Broadcasting and Web S...


In [272]:
relevant_naics = pd.read_pickle('assets/relevant_naics_df.pkl')

In [273]:
relevant_naics.head()

,query,relevant_naics
0,Home improvement store,"[444110, 444120, 444130, 444190, 444210, 44422..."
1,Diesel fuel supplier,"[424710, 424720, 424110, 424120, 424130, 42421..."
2,Church,"[813110, 813211, 813212, 813219, 813311, 81331..."
3,Farm,"[115116, 115111, 115112, 115113, 115114, 11511..."
4,Seed supplier,"[424910, 424920, 424930, 424940, 424950, 42499..."


In [274]:
max_docs = 10

## Test Results

In [275]:
def calc_pre_rec_at_n(ret_docs, reljudges, n=-1):
    """
    Calculate precision and recall at n for each query in ret_docs
    """
    
    pre_at_n, rec_at_n = {}, {}
    
    for k, v in ret_docs.items():
        if n > -1 and n <= len(ret_docs):
            s1 = set(v[:n])
        else:
            s1 = set(v)
        s2 = reljudges[k]
        precision = len(s1.intersection(s2)) / len(s1)
        recall = len(s1.intersection(s2)) / len(reljudges[k])
        pre_at_n[k] = round(precision, 3)
        rec_at_n[k] = round(recall, 3)
    return pre_at_n, rec_at_n

In [276]:
def calc_avg_pre(ret_docs, reljudges, cutoff=-1):
    """
    Calculate (mean) average precision for each query in ret_docs
    """
    
    avg_pre, mean_avg_pre = {}, None
    for k, v in ret_docs.items():
        total_rel = 0
        total = 0
        avg_prec = 0
        for i, doc in enumerate(v):
            if doc in reljudges[k] and cutoff == -1:
                total_rel += 1
                total += 1
                precision = total_rel/total
            elif doc in reljudges[k] and i+1 <= cutoff:
                total_rel += 1
                total += 1
                precision = total_rel/total
            else:
                total += 1
                precision = 0
            avg_prec += precision

        avg_pre[k] = round(avg_prec/len(reljudges[k]), 3,)
    
    mean_avg_pre = round(sum(avg_pre.values()) / len(avg_pre), 3)
        
    return avg_pre, mean_avg_pre

In [277]:
import math

def calc_NDCG_at_n(ret_docs, reljudges, n=-1, base=2):
    """
    Calculate NDCG at n for each query in ret_docs
    """
    
    ndcg = {}
    
    for k, v in ret_docs.items():
        
        counts = list(reversed([x for x in range(2,len(reljudges[k])+2)]))
        ideals = {reljudges[k][i]: counts[i] for i in range(len(reljudges[k]))}
        
        add_ons = {}
        if len(v) > len(reljudges[k]):
            for i in range(len(v)-len(reljudges[k])):
                add_ons[i] = 1
        ideals.update(add_ons)
        nums = list(map(ideals.get, v))
        
        systems = {}
        for i, doc in enumerate(v):
            if nums[i] == None:
                systems[doc] = 1
            else:
                systems[doc] = nums[i]
                
        ideal_order = {}
        if n != -1:
            for i, (key, value) in enumerate(ideals.items()):
                if i < n:
                    ideal_order[key] = value
        else:
            ideal_order = ideals
        
        add_ons = {}
        
        
        log = 0
        for i, (doc, rank) in enumerate(ideal_order.items()):
            if i >= len(v):
                break
            elif i < base:
                log += rank
            else:
                log += rank/math.log(i+1, base)

                
        system_order = {}
        if n != -1:
            for i, (key, value) in enumerate(systems.items()):
                if i < n:
                    system_order[key] = value
        else:
            system_order = systems
        
        
        
        system_log = 0
        for i, (doc, rank) in enumerate(system_order.items()):
            if i >= len(v):
                break
            elif i < base:
                system_log += rank
            else:
                system_log += rank/math.log(i+1, base)
        system_log / log
        ndcg[k] = round(system_log / log, 3)
        
    
    return ndcg

In [278]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'lemmatize'))
    ret_docs_index = tfidf_model.retrieve_docs(query_name, max_docs=max_docs, normalization='lemmatize')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [279]:
ret_docs_dic

{'home improvement store': ['442299',
  '453998',
  '453991',
  '442291',
  '453930',
  '453910',
  '453920',
  '453310',
  '445299',
  '444110'],
 'diesel fuel supplier': ['447190',
  '447110',
  '454310',
  '221112',
  '445120',
  '333618',
  '454390',
  '213115',
  '336310',
  '333611'],
 'church': ['813110',
  '337127',
  '337121',
  '337124',
  '337122',
  '337125',
  '337110',
  '423490',
  '236220',
  '327991'],
 'farm': ['115116',
  '115115',
  '115111',
  '115113',
  '115112',
  '115114',
  '114210',
  '112512',
  '112519',
  '115210'],
 'seed supplier': ['111120',
  '111110',
  '111140',
  '111150',
  '111160',
  '111199',
  '111191',
  '111219',
  '111211',
  '311911'],
 'accountant': ['541211',
  '541219',
  '541213',
  '541214',
  '813920',
  '332911',
  '332721',
  '332722',
  '332811',
  '332812'],
 'trucking company': ['484220',
  '484230',
  '484110',
  '484122',
  '484121',
  '484210',
  '551111',
  '551112',
  '551114',
  '561110'],
 'exporter': ['522293',
  '928120'

In [280]:
queries_dic

{'home improvement store': ['444110',
  '444120',
  '444130',
  '444190',
  '444210',
  '444220',
  '441110',
  '441120',
  '441210',
  '441222',
  '441228',
  '441310',
  '441320',
  '442110',
  '442210',
  '442291',
  '442299',
  '443141',
  '443142',
  '445110',
  '445120',
  '445210',
  '445220',
  '445230',
  '445291',
  '445292',
  '445299',
  '445310',
  '446110',
  '446120',
  '446130',
  '446191',
  '446199',
  '447110',
  '447190',
  '448110',
  '448120',
  '448130',
  '448140',
  '448150',
  '448190',
  '448210',
  '448310',
  '448320'],
 'diesel fuel supplier': ['424710',
  '424720',
  '424110',
  '424120',
  '424130',
  '424210',
  '424310',
  '424320',
  '424330',
  '424340',
  '424410',
  '424420',
  '424430',
  '424440',
  '424450',
  '424460',
  '424470',
  '424480',
  '424490',
  '424510',
  '424520',
  '424590',
  '424610',
  '424690',
  '424810',
  '424820',
  '424910',
  '424920',
  '424930',
  '424940',
  '424950',
  '424990',
  '423110',
  '423120',
  '423130',
 

In [281]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improvement store': 0.4,
 'diesel fuel supplier': 0.0,
 'church': 0.1,
 'farm': 1.0,
 'seed supplier': 0.0,
 'accountant': 0.4,
 'trucking company': 0.6,
 'exporter': 0.1,
 'grain elevator': 0.1,
 'popcorn store': 0.4,
 'agricultural service': 0.0,
 'warehouse': 0.4,
 'agricultural production': 1.0,
 'ranch': 1.0,
 'holding company': 0.3,
 'farm equipment supplier': 0.0,
 'store': 0.6,
 'grocery store': 0.5,
 'rice mill': 0.4,
 'food product supplier': 0.0,
 'accounting firm': 0.4,
 'produce market': 0.3,
 'pet supply store': 0.8,
 'wholesaler': 1.0,
 'produce wholesaler': 1.0,
 'distribution service': 0.4,
 'crop grower': 1.0,
 'addiction treatment center': 0.5,
 'natural good store': 0.3,
 'orchard': 1.0,
 'lumber store': 0.0,
 'mine': 1.0,
 'transportation service': 1.0,
 'investment company': 0.7,
 'fruit wholesaler': 1.0,
 'real estate agency': 1.0,
 'event venue': 0.0,
 'frozen dessert supplier': 0.8,
 'wine wholesaler importer': 0.8,
 'winery': 0.7,
 'aerospace company': 

In [282]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improvement store': 0.091,
 'diesel fuel supplier': 0.0,
 'church': 0.02,
 'farm': 0.156,
 'seed supplier': 0.0,
 'accountant': 0.082,
 'trucking company': 0.12,
 'exporter': 0.024,
 'grain elevator': 0.143,
 'popcorn store': 0.091,
 'agricultural service': 0.0,
 'warehouse': 0.571,
 'agricultural production': 0.156,
 'ranch': 0.156,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.273,
 'grocery store': 0.114,
 'rice mill': 0.055,
 'food product supplier': 0.0,
 'accounting firm': 0.082,
 'produce market': 0.068,
 'pet supply store': 0.364,
 'wholesaler': 0.141,
 'produce wholesaler': 0.141,
 'distribution service': 0.082,
 'crop grower': 0.156,
 'addiction treatment center': 0.128,
 'natural good store': 0.068,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transportation service': 0.2,
 'investment company': 0.171,
 'fruit wholesaler': 0.141,
 'real estate agency': 0.417,
 'event venue': 0.0,
 'frozen dessert supplier': 0.11,
 'wine wholesaler

In [283]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improvement store': 0.051,
 'diesel fuel supplier': 0.0,
 'church': 0.02,
 'farm': 0.156,
 'seed supplier': 0.0,
 'accountant': 0.082,
 'trucking company': 0.12,
 'exporter': 0.024,
 'grain elevator': 0.02,
 'popcorn store': 0.064,
 'agricultural service': 0.0,
 'warehouse': 0.3,
 'agricultural production': 0.156,
 'ranch': 0.156,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.238,
 'grocery store': 0.062,
 'rice mill': 0.04,
 'food product supplier': 0.0,
 'accounting firm': 0.082,
 'produce market': 0.044,
 'pet supply store': 0.359,
 'wholesaler': 0.141,
 'produce wholesaler': 0.141,
 'distribution service': 0.06,
 'crop grower': 0.156,
 'addiction treatment center': 0.11,
 'natural good store': 0.018,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transportation service': 0.2,
 'investment company': 0.114,
 'fruit wholesaler': 0.141,
 'real estate agency': 0.417,
 'event venue': 0.0,
 'frozen dessert supplier': 0.1,
 'wine wholesaler import

#### The mAP@N score is higher for TFIDF than the BoW model

In [284]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.105

In [285]:
tfidf_maps = []
for i in range(1, 11):
    tfidf_maps.append(calc_avg_pre(ret_docs_dic, queries_dic, cutoff=i)[1])

In [286]:
tfidf_maps

[0.016, 0.029, 0.042, 0.053, 0.063, 0.072, 0.081, 0.089, 0.097, 0.105]

In [287]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improvement store': 0.304,
 'diesel fuel supplier': 0.015,
 'church': 0.221,
 'farm': 0.846,
 'seed supplier': 0.015,
 'accountant': 0.631,
 'trucking company': 0.762,
 'exporter': 0.227,
 'grain elevator': 0.3,
 'popcorn store': 0.471,
 'agricultural service': 0.016,
 'warehouse': 0.621,
 'agricultural production': 0.762,
 'ranch': 0.812,
 'holding company': 1.0,
 'farm equipment supplier': 0.015,
 'store': 0.3,
 'grocery store': 0.449,
 'rice mill': 0.477,
 'food product supplier': 0.015,
 'accounting firm': 0.631,
 'produce market': 0.44,
 'pet supply store': 0.894,
 'wholesaler': 0.688,
 'produce wholesaler': 0.507,
 'distribution service': 0.085,
 'crop grower': 0.838,
 'addiction treatment center': 0.39,
 'natural good store': 0.096,
 'orchard': 0.748,
 'lumber store': 0.015,
 'mine': 0.928,
 'transportation service': 0.261,
 'investment company': 0.447,
 'fruit wholesaler': 0.659,
 'real estate agency': 0.872,
 'event venue': 0.024,
 'frozen dessert supplier': 0.699,
 'wi

## Stemming

In [288]:
docs = dict(zip(df['naics'], df['stemmed']))

In [289]:
tfidf_model = TfidfRecommenderSystem(docs)

In [290]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'stem'))
    ret_docs_index = tfidf_model.retrieve_docs(query_name, max_docs=max_docs, normalization='stem')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [291]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improv store': 0.4,
 'diesel fuel supplier': 0.0,
 'church': 0.1,
 'farm': 1.0,
 'seed supplier': 0.0,
 'account': 0.4,
 'truck compani': 0.6,
 'export': 0.0,
 'grain elev': 0.1,
 'popcorn store': 0.4,
 'agricultur servic': 0.4,
 'warehous': 0.4,
 'agricultur product': 1.0,
 'ranch': 1.0,
 'hold compani': 0.3,
 'farm equip supplier': 0.0,
 'store': 0.6,
 'groceri store': 0.4,
 'rice mill': 0.8,
 'food product supplier': 0.0,
 'account firm': 0.4,
 'produc market': 0.2,
 'pet suppli store': 0.8,
 'wholesal': 1.0,
 'produc wholesal': 1.0,
 'distribut servic': 0.4,
 'crop grower': 1.0,
 'addict treatment center': 0.5,
 'natur good store': 0.3,
 'orchard': 1.0,
 'lumber store': 0.0,
 'mine': 1.0,
 'transport servic': 1.0,
 'invest compani': 0.7,
 'fruit wholesal': 0.1,
 'real estat agenc': 1.0,
 'event venu': 0.0,
 'frozen dessert supplier': 0.8,
 'wine wholesal import': 0.8,
 'wineri': 0.7,
 'aerospac compani': 0.7,
 'cold storag facil': 0.4,
 'employ agenc': 0.4,
 'plant nurseri':

In [292]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improv store': 0.091,
 'diesel fuel supplier': 0.0,
 'church': 0.02,
 'farm': 0.156,
 'seed supplier': 0.0,
 'account': 0.082,
 'truck compani': 0.12,
 'export': 0.0,
 'grain elev': 0.143,
 'popcorn store': 0.091,
 'agricultur servic': 0.062,
 'warehous': 0.571,
 'agricultur product': 0.156,
 'ranch': 0.156,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.273,
 'groceri store': 0.091,
 'rice mill': 0.11,
 'food product supplier': 0.0,
 'account firm': 0.082,
 'produc market': 0.045,
 'pet suppli store': 0.364,
 'wholesal': 0.141,
 'produc wholesal': 0.141,
 'distribut servic': 0.082,
 'crop grower': 0.156,
 'addict treatment center': 0.128,
 'natur good store': 0.068,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transport servic': 0.2,
 'invest compani': 0.171,
 'fruit wholesal': 0.014,
 'real estat agenc': 0.417,
 'event venu': 0.0,
 'frozen dessert supplier': 0.11,
 'wine wholesal import': 0.113,
 'wineri': 0.096,
 'aerospac compani': 0.037,
 'cold

In [293]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improv store': 0.055,
 'diesel fuel supplier': 0.0,
 'church': 0.02,
 'farm': 0.156,
 'seed supplier': 0.0,
 'account': 0.082,
 'truck compani': 0.104,
 'export': 0.0,
 'grain elev': 0.02,
 'popcorn store': 0.064,
 'agricultur servic': 0.028,
 'warehous': 0.571,
 'agricultur product': 0.156,
 'ranch': 0.156,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.227,
 'groceri store': 0.051,
 'rice mill': 0.09,
 'food product supplier': 0.0,
 'account firm': 0.056,
 'produc market': 0.015,
 'pet suppli store': 0.359,
 'wholesal': 0.141,
 'produc wholesal': 0.141,
 'distribut servic': 0.036,
 'crop grower': 0.156,
 'addict treatment center': 0.104,
 'natur good store': 0.017,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transport servic': 0.2,
 'invest compani': 0.14,
 'fruit wholesal': 0.002,
 'real estat agenc': 0.417,
 'event venu': 0.0,
 'frozen dessert supplier': 0.1,
 'wine wholesal import': 0.072,
 'wineri': 0.096,
 'aerospac compani': 0.021,
 'cold s

In [294]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.102

In [295]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improv store': 0.303,
 'diesel fuel supplier': 0.015,
 'church': 0.221,
 'farm': 0.718,
 'seed supplier': 0.015,
 'account': 0.63,
 'truck compani': 0.721,
 'export': 0.026,
 'grain elev': 0.3,
 'popcorn store': 0.471,
 'agricultur servic': 0.405,
 'warehous': 0.839,
 'agricultur product': 0.143,
 'ranch': 0.873,
 'hold compani': 1.0,
 'farm equip supplier': 0.015,
 'store': 0.288,
 'groceri store': 0.423,
 'rice mill': 0.816,
 'food product supplier': 0.015,
 'account firm': 0.533,
 'produc market': 0.206,
 'pet suppli store': 0.894,
 'wholesal': 0.3,
 'produc wholesal': 0.707,
 'distribut servic': 0.076,
 'crop grower': 0.839,
 'addict treatment center': 0.377,
 'natur good store': 0.091,
 'orchard': 0.748,
 'lumber store': 0.015,
 'mine': 0.848,
 'transport servic': 0.318,
 'invest compani': 0.536,
 'fruit wholesal': 0.08,
 'real estat agenc': 0.882,
 'event venu': 0.024,
 'frozen dessert supplier': 0.7,
 'wine wholesal import': 0.48,
 'wineri': 0.827,
 'aerospac compani': 0.